In [116]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
#import ipywidgets as widgets
from IPython.display import display
import panel as pn
pn.extension('plotly')
import param
import numpy as np
import random

In [117]:
# Import data, change column header from Sex to Gender
df = pd.read_csv('../Resources/Cleaning2/merged_events_bio.csv')
df = df.rename(columns={'Sex': 'Gender'})
df.head(5)

,Athlete_ID,Name,Age,Gender,Height(cm),Weight(kg),Sport,Country_Name,Country_Code,Year,Season,isTeamSport,Medal,Result_ID
0,16809,Károly Teppert,21.0,M,NaN,NaN,Cycling Road,Hungary,HUN,1912,Summer,False,na,153233
1,16809,Károly Teppert,21.0,M,NaN,NaN,Cycling Road,Hungary,HUN,1912,Summer,True,na,153234
2,43737,Andrzej Socharski,29.0,M,173.0,72.0,Shooting,Poland,POL,1976,Summer,False,na,51745
3,50147,Nathalie Wunderlich,21.0,F,170.0,50.0,Swimming,Switzerland,SUI,1992,Summer,False,na,7860
4,50147,Nathalie Wunderlich,21.0,F,170.0,50.0,Swimming,Switzerland,SUI,1992,Summer,False,na,7870


In [118]:
# Define the medal colors
color_scale = {'Gold': 'gold', 'Silver': 'silver', 'Bronze': 'brown'}

In [119]:
# Define the function to filter the data 
def filter_data(df, gender, sport=None, country=None):
    filtered_df = df.copy()
    if gender != 'Both':
        filtered_df = filtered_df[filtered_df['Gender'] == gender]
    if sport != 'ALL':
        filtered_df = filtered_df[filtered_df['Sport'] == sport]
    if country != 'ALL':
        filtered_df = filtered_df[filtered_df['Country_Name'] == country]
    return filtered_df

In [120]:
# Define a function to update the scatter plot based on the dropdown menus
def update_scatter_plot(gender, sport, country):
    # Filter the data 
    filtered_df = filter_data(df, gender, sport, country)
    
    # Create the scatter plot
    fig = px.scatter(filtered_df, x='Weight(kg)', y='Height(cm)', color='Medal', symbol='Gender', color_discrete_map=color_scale,
                     symbol_map={'M': 'square', 'F': 'circle'}, title=f'Olympic Medalists Height & Weight')
    
    # Set the axis labels
    fig.update_layout(xaxis_title='Weight (kg)', yaxis_title='Height (cm)')
    
    # Return the plot
    #fig.show()
    return pn.pane.Plotly(fig)

In [121]:
scatter_plot = go.Figure()


In [122]:
# Define the function to update the medal bar chart
def update_bar_chart(gender, sport, country):
    filtered_df = filter_data(df, gender, sport, country)
    filtered_df = filtered_df[filtered_df['Age'].notnull()] # Filter out rows where Age is null
    filtered_df = filtered_df[~(filtered_df['Medal'] == 'na')]  # Filter out rows where no medal was received
    filtered_df['Age'] = filtered_df['Age'].astype(int) # Convert Age column to integer
    medal_counts = pd.pivot_table(filtered_df, index=['Age'], columns=['Medal'], values='Name', aggfunc='count', fill_value=0)

    # Add a new column to medal_counts with the total count for each age
    medal_counts['Total'] = medal_counts['Gold'] + medal_counts['Silver'] + medal_counts['Bronze']

    age_order = sorted(filtered_df['Age'].unique())
    fig = px.bar(medal_counts, x=medal_counts.index, y=['Gold', 'Silver', 'Bronze'], barmode='stack',
                 color_discrete_map={'Gold': 'gold', 'Silver': 'silver', 'Bronze': 'brown'},
                 labels={'value': 'Medal Count', 'variable': 'Medal'},
                 category_orders={'x': age_order},
                 hover_data={'Total': True}) # Add the total count to the hover label

    fig.update_layout(title='Medal Count by Age', xaxis_title='Age', yaxis_title='Medal Count', legend_title='Medal')
    
    # Update x-axis ticks based on number of age groups
    num_age_groups = len(age_order)
    if num_age_groups <= 20:
        fig.update_xaxes(tickmode='array', tickvals=medal_counts.index, ticktext=medal_counts.index)
    elif num_age_groups > 20 and num_age_groups <= 50:
        fig.update_xaxes(tickmode='linear', dtick=5)
    else:
        fig.update_xaxes(tickmode='linear', dtick=10)
    
    # Update the Panel component
    #fig.show()
    return pn.pane.Plotly(fig)

In [123]:
# Define dropdown menus, alphabetize sports and countries
gender_dropdown = pn.widgets.Select(
    options=['Both', 'M', 'F'],
    value='Both',
    name='Gender:')

sports = ["ALL"] + sorted(df['Sport'].unique().tolist())
sports.sort()
sport_dropdown = pn.widgets.Select(options=sports, value="ALL", name="Sport:")

countries = ["ALL"] + sorted(df['Country_Name'].unique().tolist())
countries.sort()
country_dropdown = pn.widgets.Select(options=countries, value="ALL", name='Country:')

# Define the callback function
def update_plots():
    gender = pn.state.curdoc.get('gender', None)
    sport = pn.state.curdoc.get('sport', None)
    country = pn.state.curdoc.get('country', None)
    new_gender = gender_dropdown.value
    new_sport = sport_dropdown.value
    new_country = country_dropdown.value
    if new_gender != gender or new_sport != sport or new_country != country:
        scatter = update_scatter_plot(new_gender, new_sport, new_country)
        bar = update_bar_chart(new_gender, new_sport, new_country)
        scatter_panel.object = scatter
        bar_panel.object = bar
        pn.state.curdoc['gender'] = new_gender
        pn.state.curdoc['sport'] = new_sport
        pn.state.curdoc['country'] = new_country
        
# Register the callback function
gender_dropdown.param.watch(update_plots, 'value')
sport_dropdown.param.watch(update_plots, 'value')
country_dropdown.param.watch(update_plots, 'value')

def reset_dropdowns(event):
    # reset dropdown menus to their original values
    gender_dropdown.value = 'Both'
    sport_dropdown.value = 'ALL'
    country_dropdown.value = 'ALL'
    update_plots()
    
# Add a Reset button
reset_button = pn.widgets.Button(name='Reset', button_type='primary')
reset_button.on_click(reset_dropdowns)

# Display the dropdown menus
controls = pn.Row(gender_dropdown, sport_dropdown, country_dropdown, reset_button)

# Create the scatter plot and bar chart
scatter = update_scatter_plot(gender_dropdown.value, sport_dropdown.value, country_dropdown.value)
bar = update_bar_chart(gender_dropdown.value, sport_dropdown.value, country_dropdown.value)

scatter_panel = pn.panel(scatter)
bar_panel = pn.panel(bar)

dashboard = pn.Column(
    '## Olympic Medaling',
    controls,
    #dropdown_row,
    pn.Row(scatter_panel, bar_panel, sizing_mode='stretch_both')
)

pn.serve(dashboard)

Launching server at http://localhost:52714
